In [10]:
import pandas as pd
import openpyxl
from pandas import ExcelWriter

In [11]:
%store -r tarjetas
%store -r bancos

In [12]:
tarjeta_mastercard_debit = tarjetas.loc [tarjetas['Cod Marca Producto']== 'Master Debit'] #Filtro por la Tarjeta Mastercard Debit
tarjeta_mastercard_debit = tarjeta_mastercard_debit.loc[:, (tarjeta_mastercard_debit != 0).any(axis=0)]

In [13]:
tarjeta_mastercard_debit["RETENCION ING.BRUTOS SIRTAC"] = tarjeta_mastercard_debit[
                        ["RETENCION ING.BRUTOS SIRTAC", "CARGO POSNET WWW.FISERV.COM.AR"]].sum(axis=1, min_count=1) #junto columnas relacionadas en una
tarjeta_mastercard_debit["TOTAL DEDUCCIONES"] = tarjeta_mastercard_debit[
                        ["TOTAL DEDUCCIONES", "TOTAL PAGOS DE COMERCIOS"]].sum(axis=1, min_count=1) #junto columnas relacionadas en una
tarjeta_mastercard_debit = tarjeta_mastercard_debit.reindex(columns=['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio',
       'Nro Liquidacion','VENTAS C/DESCUENTO CONTADO', 'ARANCEL', 'IVA CRED.FISC.COMERCIO S/ARANC 21,00%',
       'RETENCION IVA','PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS','RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC','PERCEPCION IVA R.G. 2408   3,00 %', 'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION'])
         #reordeno columnas, aca ya quedan en el orden que es necesario

tarjeta_mastercard_debit.insert(7, 'Iva 21', 
                                tarjeta_mastercard_debit['IVA CRED.FISC.COMERCIO S/ARANC 21,00%']/tarjeta_mastercard_debit['ARANCEL'])
                                #agrego columna de control para IVA %2
tarjeta_mastercard_debit = tarjeta_mastercard_debit.assign(Diferencia = tarjeta_mastercard_debit.iloc[:, 5:7].sum(axis=True) 
                         + tarjeta_mastercard_debit.iloc[:, 8:14].sum(axis=True)
                         - tarjeta_mastercard_debit['TOTAL DEDUCCIONES']) #agrego columna de control final
tarjeta_mastercard_debit = tarjeta_mastercard_debit.round(2)


In [14]:
sum_columns = ['Nro Liquidacion', 'VENTAS C/DESCUENTO CONTADO', 'ARANCEL', 'IVA CRED.FISC.COMERCIO S/ARANC 21,00%', 'Iva 21', 'RETENCION IVA',
               'PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS', 'RETENCION ING.BRUTOS', 'RETENCION ING.BRUTOS SIRTAC', 
               'PERCEPCION IVA R.G. 2408   3,00 %', 'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION', 'Diferencia']
max_columns = ['Nro Liquidacion']

group = {col: ('max' if col in max_columns else 'sum') for col in sum_columns}

tarjeta_mastercard_debit=tarjeta_mastercard_debit.groupby(by=['Cod Marca Producto', 'Cod Comercio']).agg(group)

In [16]:
tarjeta_mastercard_debit

Desc Concepto Clearing,Cod Marca Producto,Cod Comercio,Nro Liquidacion,VENTAS C/DESCUENTO CONTADO,ARANCEL,"IVA CRED.FISC.COMERCIO S/ARANC 21,00%",RETENCION IVA,PER B.A.I.BR.DN.01/04,RETENCION IMP.GANANCIAS,RETENCION ING.BRUTOS,RETENCION ING.BRUTOS SIRTAC,"PERCEPCION IVA R.G. 2408 3,00 %",TOTAL DEDUCCIONES,TOTAL LIQUIDACION
0,Master Debit,4547438,149303,71080394.3,568631.23,119412.57,352558.81,3980.42,352558.81,1396132.92,606597.74,17063.42,3416935.92,67701412.95


In [15]:
tarjeta_mastercard_debit.drop(['Diferencia', 'Iva 21'], inplace=True, axis=1)
tarjeta_mastercard_debit.reset_index(inplace=True)

#Aca termina. Hay que exportar la tabla a un Excel

In [17]:
writer = ExcelWriter(r'C:\Users\Juan M Cabral\Desktop\Liquidaciones tarjetas\11-22\Masterdebit1122.xlsx')
tarjeta_mastercard_debit.to_excel(writer, 'Hoja de datos', index=False)
writer.save()

C:\Users\Juan M Cabral\AppData\Local\Temp\ipykernel_12620\99857620.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
